# 간단한 Transformer 구현해보기
- IMDB 데이터셋을 가지고, review 에 대해서 긍정인지 부정인지를 판별하는 모델을 만든다. 

In [1]:
import torch
import torch.nn as nn
import math
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

## Tokenizer 준비

In [3]:
ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load(
    "huggingface/pytorch-transformers", "tokenizer", "bert-base-uncased"
)

Using cache found in /Users/joyuiyeong/.cache/torch/hub/huggingface_pytorch-transformers_main
/Users/joyuiyeong/.pyenv/versions/deeplearning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## IMDB 에 대한 DataLoader 준비

In [4]:
max_len = 400


def collate_imdb(batch):
    texts, labels = [], []
    for row in batch:
        texts.append(row["text"])
        labels.append(row["label"])

    texts = torch.LongTensor(
        tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids
    )
    labels = torch.LongTensor(labels)
    return texts, labels


train_data_loader = DataLoader(
    ds["train"], batch_size=64, shuffle=True, collate_fn=collate_imdb
)
test_data_loader = DataLoader(
    ds["test"], batch_size=64, shuffle=False, collate_fn=collate_imdb
)

## Transformer 의 Encoder 구조

```mermaid
flowchart TB
    Input[("Input Tokens")]
    Embed["Embedding Layer"]
    PosEnc["Positional Encoding"]
    
    subgraph "Encoder Layer (Repeated N times)"
        direction TB
        subgraph "Multi-Head Attention"
            direction LR
            Q["Q Linear"]
            K["K Linear"]
            V["V Linear"]
            QK{{"Q * K^T / √dk"}}
            Softmax["Softmax"]
            AttOut["Attention Output"]
        end
        
        Add1(("+ Add"))
        Norm1["Layer Norm"]
        
        FF["Feed Forward Network"]
        
        Add2(("+ Add"))
        Norm2["Layer Norm"]
    end
    
    Output[("Output")]
    
    Input --> Embed
    Embed --> PosEnc
    PosEnc --> Q & K & V
    Q & K --> QK
    QK --> Softmax
    Softmax & V --> AttOut
    
    AttOut --> Add1
    PosEnc -.-> Add1
    Add1 --> Norm1
    Norm1 --> FF
    FF --> Add2
    Norm1 -.-> Add2
    Add2 --> Norm2
    Norm2 --> Output
```

## Self-Attention 구현

- Shape이 $(S, D)$인 embedding $x$가 주어졌을 때, self-attention은 다음과 같이 계산합니다:

```mermaid
flowchart TB
    Input[("Input Sequence")]
    
    subgraph "Self-Attention"
        direction TB
        QProj["Query Projection"]
        KProj["Key Projection"]
        VProj["Value Projection"]
        
        MatMul1{{"Matrix Multiplication"}}
        Scale[/"Scale (÷ √dk)"\]
        Mask["Apply Mask (optional)"]
        Softmax["Softmax"]
        MatMul2{{"Matrix Multiplication"}}
    end
    
    Output[("Attention Output")]
    
    Input --> QProj & KProj & VProj
    QProj --> MatMul1
    KProj --> KT["Transpose"]
    KT --> MatMul1
    MatMul1 --> Scale
    Scale --> Mask
    Mask --> Softmax
    Softmax --> MatMul2
    VProj --> MatMul2
    MatMul2 --> Output
    
    style Input fill:#f9f,stroke:#333,stroke-width:4px
    style Output fill:#bbf,stroke:#333,stroke-width:4px
    style QProj fill:#fdd
    style KProj fill:#dfd
    style VProj fill:#ddf
    style MatMul1 fill:#ffd
    style MatMul2 fill:#ffd
    style Scale fill:#eff
    style Mask fill:#ffe
    style Softmax fill:#eef
```

$$
\begin{align*} Q, K, V &= xW_q, xW_k, xW_v \in \mathbb{R}^{S \times D},\\ A &= \textrm{Softmax}\left(\frac{QK^T}{\sqrt{D}}, \textrm{dim=1}\right) \in \mathbb{R}^{S \times S}, \\ \hat{x}&=AV W_o \in \mathbb{R}^{S \times D}. \end{align*}
$$

- 여기서 $W_q, W_k, W_v, W_o \in \mathbb{R}^{D \times D}$는 MLP에서 사용하는 weight matrix와 동일한 parameter들입니다. 
- 보시다시피 $Q$를 자기자신 $x$로 부터 뽑은 것을 제외하면 sequence-to-sequence와 동일합니다. 자기자신과 attention을 계산하여 처리하기 대문에 self-attention이라고 부릅니다.

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        # mask 는 실제 attention 계산에서 padding token 을 무시하기 위해 제공되는 tensor
        q, k, v = self.wq(x), self.wk(x), self.wv(x)
        score = torch.matmul(q, k.transpose(-2, -1))
        score = score / math.sqrt(self.d_model)

        if mask is not None:
            # -1e9 는 매우 작은 값으로, softmax 를 거치게 되면 0에 가까워져서 weight sum 과정에서 padding token 은 무시할 수 있게 됩니다.
            score = score + (mask * -1e9)

        score = self.softmax(score)

        result = torch.matmul(score, v)
        result = self.dense(result)
        return result

## 간단한 Transformer Layer
- Self-Attention 층과 Feed-Forward 층만 있는 Transformer Layer 를 정의합니다.

In [ ]:
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, d_model, dff):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        self.sa = SelfAttention(input_dim, d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model),
        )

    def forward(self, x, mask):
        x = self.sa(x, mask)
        x = self.ffn(x)
        return x

## Positional Encoding
- Scaled Dot Product 인 Self-Attention 만 하면, token 의 위치 정보를 반영하지 못합니다.
- 그래서 위치 정보도 넣어주기 위해 Positional Encoding 을 진행합니다.
- `nn.Embedding`에서 나온 embedding 들에 다음과 같은 positional encoding 이라는 값을 더해줘 순서 정보를 주입합니다 

$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

- 여기서 $(S, D)$는 입력 embedding $x$의 shape입니다. 

- 결과적으로 다음과 같이 순서 정보를 주입합니다:

$$
x_{\textrm{positional}} = x + PE.
$$

- Transformer의 positional encoding을 주기함수를 쓰고, 각 차원마다 다른 주기함수를 쓰는 것 같습니다. 왜 이렇게 주기함수를 빈번하게 사용하는건가요?
    - 위와 같이 positional encoding을 설정한 이유는 다음과 같이 정리할 수 있습니다.
        1. **Bound된 positional encoding 값:** 주기함수를 쓰면 값들이 bound되기 때문에 아주 큰 값이 embedding에 더해지는 것을 방지할 수 있습니다.
        2. **위치마다 다른 positional encoding 값:** 기본적으로 positional encoding은 token 위치마다 다른 값을 가져야 합니다. 차원마다 다른 주기함수를 사용하여 이를 보장해줍니다.
        3. **$S$와 무관한 positional encoding 값:** 우리가 궁금한건 token 사이의 상대적인 위치 정보이지, 절대적인 정보가 아닙니다. 그래서 $S$와 무관한 positional encoding이 필요합니다.

- 결과적으로 만들어진 positional encoding $PE$를 가지고 다음과 element-wise 덧셈 연산을 사용하여 순서 정보를 주입합니다.
- 이렇게 위치 정보를 미리 계산해서 넣으면, 이 정보에 대해서는 학습을 진행하지 않고 계산된 값을 사용합니다.

In [10]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10_000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, None], np.arange(d_model)[None, :], d_model
    )
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)

## 모델 정의
- 위에서 정의한 SelfAttention, TransformerLayer, positional_encoding 을 사용하여, model 을 정의합니다. 

In [6]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, dff):
        super().__init__()

        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.dff = dff

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.parameter.Parameter(
            positional_encoding(max_len, d_model), requires_grad=False
        )
        self.layers = nn.ModuleList(
            [TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)]
        )
        self.classification = nn.Linear(d_model, 1)

    def forward(self, x):
        mask = x == tokenizer.pad_token_id
        mask = mask[:, None, :]
        seq_len = x.shape[1]

        x = self.embedding(x)
        x = x * math.sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len]

        for layer in self.layers:
            x = layer(x, mask)

        x = x[:, 0]
        x = self.classification(x)
        return x

In [7]:
from torch.optim import Adam

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

lr = 0.001
model = TextClassifier(vocab_size=len(tokenizer), d_model=32, n_layers=2, dff=32).to(
    device
)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [8]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(m, dataloader):
    cnt = 0
    acc = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        preds = m(inputs)
        # preds = torch.argmax(preds, dim=-1)
        preds = (preds > 0).long()[..., 0]

        cnt += labels.shape[0]
        acc += (labels == preds).sum().item()

    return acc / cnt

In [9]:
n_epochs = 50

for epoch in range(n_epochs):
    total_loss = 0.0
    model.train()
    for data in train_data_loader:
        model.zero_grad()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device).float()

        preds = model(inputs)[..., 0]
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

    with torch.no_grad():
        model.eval()
        train_acc = accuracy(model, train_data_loader)
        test_acc = accuracy(model, test_data_loader)
        print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 224.93581557273865
=========> Train acc: 0.788 | Test acc: 0.756
Epoch   1 | Train Loss: 171.86157739162445
=========> Train acc: 0.841 | Test acc: 0.791
Epoch   2 | Train Loss: 146.20741969347
=========> Train acc: 0.868 | Test acc: 0.805
Epoch   3 | Train Loss: 127.01036885380745
=========> Train acc: 0.892 | Test acc: 0.808
Epoch   4 | Train Loss: 109.9486108198762
=========> Train acc: 0.911 | Test acc: 0.805
Epoch   5 | Train Loss: 93.10522639751434
=========> Train acc: 0.933 | Test acc: 0.809
Epoch   6 | Train Loss: 76.56934222206473
=========> Train acc: 0.947 | Test acc: 0.805
Epoch   7 | Train Loss: 66.30307236686349
=========> Train acc: 0.959 | Test acc: 0.806
Epoch   8 | Train Loss: 53.81126401014626
=========> Train acc: 0.969 | Test acc: 0.807
Epoch   9 | Train Loss: 42.767592184245586
=========> Train acc: 0.977 | Test acc: 0.802
Epoch  10 | Train Loss: 33.90505462652072
=========> Train acc: 0.982 | Test acc: 0.803
Epoch  11 | Train Loss: 26.080